In [1]:
%matplotlib inline
import sparse_ternary_networks.Tools as T
import sparse_ternary_networks.loadLab as load
import numpy as np
import sparse_ternary_networks.MLSTC as MLSTC
import matplotlib.pyplot as plt
##
import torch
from torch import nn
from torch.autograd import Variable
import sparse_ternary_networks.PyTorcher_MLSTC as STNetTorcher
import RRQ.Tools as Tools

In [2]:
# Initializing the parameters:
n = 512      # dimension
m = 512       # dimension of the codes
N_train = 10000     # number of samples
N_test = 10000     # number of samples
k = 2         # number of non-zero elements of the STC at each stage.
L = 10        # number of layer units of the network.

In [3]:
# Torch hyper-params:
dtype = torch.FloatTensor
num_epoch = 10    
learning_rate = 1e-3  
num_batch = 100
weight_decay = 0

In [4]:
numRep = 1
RhoList = [0.5]
#RhoList = [0,0.5,0.99]
MethodList = ['STNetRandom','STNetMLSTC','Random','MLSTC']
#MethodList = ['RQ']
for RhoInd,Rho in enumerate(RhoList):
    for Method in MethodList:
        exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_train = np.zeros((numRep,L+1))')
        exec('dist_' + Method + '_Rho' + str(int(Rho*100)) + '_test = np.zeros((numRep,L+1))')
        #
        exec('rate_' + Method + '_Rho' + str(int(Rho*100)) + ' = np.zeros((numRep,L+1))')

In [5]:
MethodList = ['STNetMLSTC','MLSTC']
for RepInd in range(numRep):
    print('.......................................  Repetition = ', RepInd + 1)
    for RhoInd, Rho in enumerate(RhoList):
        print('....................  Rho = ', Rho)
        F_train, F_test = Tools.data_generator(n, N_train, N_test, 'AR1', SourceParam=Rho)
        norm_train = np.divide(np.linalg.norm(F_train) ** 2, np.prod(F_train.shape))
        norm_test = np.divide(np.linalg.norm(F_test) ** 2, np.prod(F_test.shape))
        for MethodInd, Method in enumerate(MethodList):
            print(' ############   ' + Method + '     A L G O R I T H M   ################')

            ###############################################
            # Pre-training:
            print('Pre-training:')
            if 'Random' in Method:
                Learner = 'Random'
            elif 'MLSTC' in Method:
                Learner = 'SuccessivePCA'
            ##    
            obj_pretrain = MLSTC.BaseLearner(k, L, m=m, Learner=Learner)
            _, _, _ = obj_pretrain.run(F_train)
            #
            if 'Random' in Method:
                exec('dist_' + 'Random' + '_Rho' + str(
                    int(Rho * 100)) + '_train[RepInd,:] = obj_pretrain.distortion/norm_train')
                exec('rate_' + 'Random' + '_Rho' + str(int(Rho * 100)) + '[RepInd,:] = obj_pretrain.rate')

            elif 'MLSTC' in Method:
                exec('dist_' + 'MLSTC' + '_Rho' + str(
                    int(Rho * 100)) + '_train[RepInd,:] = obj_pretrain.distortion/norm_train')
                exec('rate_' + 'MLSTC' + '_Rho' + str(int(Rho * 100)) + '[RepInd,:] = obj_pretrain.rate')

            # testing the pre-training:
            obj_pretest = MLSTC.fwdPass(obj_pretrain.params, k, ternaryProbMap=obj_pretrain.prob_z)
            _, _, _ = obj_pretest.run(F_test)
            if 'Random' in Method:
                exec('dist_' + 'Random' + '_Rho' + str(
                    int(Rho * 100)) + '_test[RepInd,:] = obj_pretest.distortion/norm_test')
            elif 'MLSTC' in Method:
                exec('dist_' + 'MLSTC' + '_Rho' + str(
                    int(Rho * 100)) + '_test[RepInd,:] = obj_pretest.distortion/norm_test')

            if 'STNet' not in Method:
                continue
            ###############################################
            # Fine-tuning:
            print('Fine-tuning: ')
            params = obj_pretrain.params
            #############################
            ## Training to fine-tune:
            obj_bp_train = STNetTorcher.fwdPass(params, k, nlinStrategy='KBest', beta_shape='scalar')
            optimizer = torch.optim.Adam(obj_bp_train.parameters(), lr=learning_rate, weight_decay=weight_decay)
            # ################################
            F_torch = Variable(torch.from_numpy(np.copy(F_train)).type(dtype))
            counter = 0
            for epoch in range(num_epoch):
                rand_perm = np.random.permutation(N_train)
                batch_map = []
                for iter in range(num_batch):
                    batch_map.append(
                        rand_perm[np.arange(iter * int(N_train / num_batch), (iter + 1) * int(N_train / num_batch))])
                for iter in range(num_batch):  # gives batch data
                    batch_map_current = batch_map[iter]
                    F_minibatch = F_torch[:, batch_map_current]
                    F_chapeau = obj_bp_train.forward(F_minibatch)

                    loss = (F_minibatch - F_chapeau).pow(2).mean()
                    if iter % 10 == 0:
                        print('Epoch: ', epoch + 1, '| train loss: %.4f' % loss.data[0])
                    # Zero gradients, perform a backward pass, and update the weights.
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    counter += 1
                    # -------------
            for l in range(L):
                obj_bp_train.params[l]['A'] /= np.linalg.norm(obj_bp_train.params[l]['A'], axis=1).reshape(-1,
                                                                                                           1)

            # Re-adjusting beta:
            obj_beta = MLSTC.BaseLearner(k, L, m=m, Learner='SuccessivePCA')
            obj_beta.reRun_reweightUpdate(F_train, obj_bp_train.params)
            ###### Re-running on the final network on train set:
            obj_beta_train = MLSTC.fwdPass(obj_beta.params, k)
            _ = obj_beta_train.run(F_train)
            exec('dist_' + Method + '_Rho' + str(
                int(Rho * 100)) + '_train[RepInd,:] = obj_beta_train.distortion/norm_train')
            exec('rate_' + Method + '_Rho' + str(int(Rho * 100)) + '[RepInd,:] = obj_beta_train.rate')
            ###### Re-running on the final network on test set:
            obj_beta_test = MLSTC.fwdPass(obj_beta.params, k)
            _ = obj_beta_test.run(F_test)
            exec('dist_' + Method + '_Rho' + str(
                int(Rho * 100)) + '_test[RepInd,:] = obj_beta_test.distortion/norm_test')


.......................................  Repetition =  1
....................  Rho =  0.5
 ############   STNetMLSTC     A L G O R I T H M   ################
Pre-training:
 ################## Starting to learn network parameters:  ##################
layer-units:
** 1 **** 2 **** 3 **** 4 **** 5 **** 6 **** 7 **** 8 **** 9 **** 10 **
Finished learning network parameters:
 Running the network:
 ##################  Running the network:  ##################
layer-units:
** 1 **** 2 **** 3 **** 4 **** 5 **** 6 **** 7 **** 8 **** 9 **** 10 **
Finished running the network.
Fine-tuning: 


/home/sssohrab/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:69: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch:  1 | train loss: 0.5636
Epoch:  1 | train loss: 0.5621
Epoch:  1 | train loss: 0.5708
Epoch:  1 | train loss: 0.5697
Epoch:  1 | train loss: 0.5691
Epoch:  1 | train loss: 0.5659
Epoch:  1 | train loss: 0.5716
Epoch:  1 | train loss: 0.5714
Epoch:  1 | train loss: 0.5706
Epoch:  1 | train loss: 0.5746
Epoch:  2 | train loss: 0.5185
Epoch:  2 | train loss: 0.5087
Epoch:  2 | train loss: 0.5153
Epoch:  2 | train loss: 0.5187
Epoch:  2 | train loss: 0.5251
Epoch:  2 | train loss: 0.5283
Epoch:  2 | train loss: 0.5216
Epoch:  2 | train loss: 0.5241
Epoch:  2 | train loss: 0.5347
Epoch:  2 | train loss: 0.5381
Epoch:  3 | train loss: 0.4750
Epoch:  3 | train loss: 0.4712
Epoch:  3 | train loss: 0.4780
Epoch:  3 | train loss: 0.4930
Epoch:  3 | train loss: 0.4962
Epoch:  3 | train loss: 0.4913
Epoch:  3 | train loss: 0.5063
Epoch:  3 | train loss: 0.4928
Epoch:  3 | train loss: 0.5128
Epoch:  3 | train loss: 0.5057
Epoch:  4 | train loss: 0.4391
Epoch:  4 | train loss: 0.4413
Epoch:  

In [6]:
curve_dict = {}
for RhoInd,Rho in enumerate(RhoList):
    for MethodInd,Method in enumerate(MethodList):
        exec('x = np.mean(rate_' +  Method + '_Rho' + str(int(Rho*100)) + ',axis=0)')
        exec('y = np.mean(dist_' +  Method + '_Rho' + str(int(Rho*100)) + '_train,axis=0)')
        exec("curve_dict['Rho" + str(int(Rho*100))+ '_n' + str(n)+ '_m' + str(m) +"_DR_"+ Method + "_train'] = (x,y)") 
        exec('y = np.mean(dist_' +  Method + '_Rho' + str(int(Rho*100)) + '_test,axis=0)')
        exec("curve_dict['Rho" + str(int(Rho*100))+ '_n' + str(n)+ '_m' + str(m) +"_DR_"+ Method + "_test'] = (x,y)") 

In [7]:
# # Saving the results for PGFplott:
# PGF_path = 'PGF/dat/BP/'
# ExpName = 'SyntheticAR1_BP'
# #
# for curve_name, curve_xy in curve_dict.items():
#     fname = ExpName + '_' + curve_name + '.dat'
#     print(fname)
#     x = curve_xy[0].astype(float).reshape(-1)
#     y = curve_xy[1].astype(float).reshape(-1)
#     np.savetxt(PGF_path + fname, np.transpose([x,y]), fmt='%8f', delimiter='   ')